# Regularization

In a broader sense, the term regularization is used to indicate measures taken against overfitting, including early termination. However, most commonly, regularization refers to an 'embedded method' that weights or penalizes the features according to their importance.

From the very beginning of polynomial regression, it was observed that when these functions overfit, quite often we see bizar values for some of the coefficients. Thus one of the first ideas to prevent overfitting was to prevent the learned parameters from taking on extreme values. Two very popular methods add the sum of the parameters $\theta$ as a penalty to the cost function, so that these parameters are minimized while learning the model.

`Lasso regression` (L1) adds the sum of abolute values in theta. `Ridge regression` (L2) adds the sum of squared values in $\theta$. They behave differently, as Lasso really tries to set coefficients to zero, really eliminating features, while Ridge does not really eliminate but rather downweights coefficients. In SKLearn, Ridge regression is called L2 and Lasso regression is called L1. 

At the end of the Logistic regression cost function, you can see how the penalty for Lasso Regression is added. $\lamba$ is a small number, usually between 0 and 1, that controls the amount of regression used. More regression means that coefficients are pushed more to zero. Ridge regression is the same, but with the penalty squared.

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m -y_i \cdot logit(\theta^T \cdot x_i) + (1 - y_i) \cdot (1 - logit( \theta^T \cdot x_i ) ) + \frac{\lambda}{m} \sum_j | \theta_j | $$ 

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, mutual_info_classif, f_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from matplotlib import pyplot as plt
from math import sqrt
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score

Load the houses dataset. Because we want to focus on features, we simply remove records with missing values. Note that you should normall do this with more care! We also convert categories to dummy variables and create a train and test set.

In [2]:
df = pd.read_csv('/data/datasets/bank-additional-full.csv', delimiter=';')

In [3]:
df = df.drop(columns=df.columns[df.isnull().any()])

In [4]:
df.y = (df.y == 'yes').astype(int)

In [5]:
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns, drop_first=True)

In [6]:
all_features = df.columns.drop('y')

In [7]:
train_X, valid_X, train_y, valid_y = train_test_split(df[all_features], df.y, test_size=0.2)

In [8]:
train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.2)

There are 26k rows in the training set and 53 features.

In [9]:
train_X.shape

(26360, 53)

In [10]:
#poly = PolynomialFeatures(degree=1)
#train_X = poly.fit_transform(train_X)
#valid_X = poly.fit_transform(valid_X)
#test_X = poly.fit_transform(test_X)

scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
valid_X = scaler.transform(valid_X)
test_X = scaler.transform(test_X)

Let's train a model on all of the training data.

In [11]:
model = LogisticRegression(penalty='none')
model.fit(train_X, train_y)
pred_y = model.predict(valid_X)
f1_score(valid_y, pred_y)

0.5268225584594223

### Lasso regression

For Lasso regression we also need a different solver. You should see that the predictions (F1 score) improve and that some coefficients are set to zero. `C` is a hyperparameter that controls the amount of regularization, or more specific is $\frac{1}{\lambda}$. You should tune `C`.

In [41]:
model = LogisticRegression(penalty='l1', 
        solver='liblinear', C=0.1)
model.fit(train_X, train_y)
pred_y = model.predict(valid_X)
f1_score(valid_y, pred_y)

0.5346127484578478

In [37]:
model.coef_

array([[-7.56893246e-03,  1.20820228e+00, -9.79246002e-02,
        -2.09651319e-01, -6.78745061e-02, -1.46954427e+00,
         6.81015002e-01,  1.07010400e-01,  0.00000000e+00,
         0.00000000e+00, -1.03760730e-01, -2.20992713e-02,
         0.00000000e+00, -2.81869772e-02,  4.20911320e-02,
        -4.63288576e-02, -5.52724995e-02,  1.41023371e-02,
         0.00000000e+00,  4.15921240e-03,  5.69697078e-03,
         0.00000000e+00,  1.09973548e-02, -3.32658107e-03,
         2.58117644e-02, -3.85145390e-02,  0.00000000e+00,
         9.67237217e-03,  3.41600197e-02,  8.36989461e-02,
         6.81214887e-04, -1.09314029e-01,  0.00000000e+00,
        -6.86432343e-05,  0.00000000e+00, -7.08047278e-03,
        -5.31582195e-02, -1.80988306e-01,  2.01391915e-01,
         1.19735093e-02,  8.41853061e-02, -2.97301219e-02,
         2.01470986e-01, -2.66071123e-01, -6.83567826e-02,
         3.06922530e-02,  1.68006797e-02, -5.88015011e-02,
         4.91825548e-03,  2.18405703e-02,  4.93424121e-0

### Ridge regression

For Ridge regression we can use the default solver. Note that ridge regression is the default for Logistic Regression. You still need to tune `C`.

In [38]:
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(train_X, train_y)
pred_y = model.predict(valid_X)
f1_score(valid_y, pred_y)

0.5326012354152367

In [39]:
model.coef_

array([[-0.01473288,  1.21526303, -0.11332017, -0.21572349, -0.07536412,
        -1.35888082,  0.55634626,  0.07978823,  0.21189892, -0.24756866,
        -0.10986534, -0.02976094, -0.0020757 , -0.03656643,  0.04887141,
        -0.05482977, -0.0632065 ,  0.01402992, -0.00513395,  0.00841193,
         0.01126673,  0.00181082,  0.01265381, -0.00809736,  0.04184178,
        -0.03320591,  0.00865741,  0.01313419,  0.04875174,  0.10104965,
         0.01206419, -0.11655013, -0.0247326 , -0.00667164,  0.00426174,
        -0.00667164, -0.06148709, -0.18298835,  0.1843973 ,  0.00857058,
         0.08411692, -0.01520779,  0.19387046, -0.28376377, -0.0965048 ,
         0.01766226,  0.00156738, -0.05405738,  0.01926249,  0.03494664,
         0.06331152,  0.11052233,  0.15555402]])

SKLearn has a generic method `SelectKBest` to rank features according to a feature selection criterium. Since this is a classification problem, we should use one of the classification criteria. The criterium we use is `mutual_info_regression`. For this a decision tree is learned to divide the records over thresholds of the given feature. The mutual information is highest when the decision boundary in the dataset results in less entropy, in other words more homogene classes. Another criterium you can try is `f_classif` which uses the Annova F-test to rank the features.

In [44]:
model = LogisticRegression(penalty='none')
rfe = RFE(model, n_features_to_select=20)
rfe.fit(train_X, train_y)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


RFE(estimator=LogisticRegression(penalty='none'), n_features_to_select=20)

In [46]:
all_features[rfe.get_support()]

Index(['duration', 'campaign', 'pdays', 'emp.var.rate', 'cons.price.idx',
       'euribor3m', 'nr.employed', 'job_blue-collar', 'default_unknown',
       'default_yes', 'contact_telephone', 'month_aug', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'day_of_week_mon',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [47]:
model.fit(rfe.transform(train_X), train_y)
f1_score(valid_y, model.predict(rfe.transform(valid_X)))

0.5259165613147914

We then learn and validate $k$ models that hold the best $k$ features. We use the F1-score here, whch is de harmonic mean between recall and precision, since it is easier to compare the models with just one number.

In [ ]:
f1v = []
f1t = []
for k in range(1, train_X.shape[1]):
    # use only the k best features
    
    model = LogisticRegression(penalty='none')
    rfe = RFE(model, n_features_to_select=k)
    rfe.fit(train_X, train_y)
    
    # fit the model and run the evaluation
    model.fit(rfe.transform(train_X), train_y)
    f1v.append(f1_score(valid_y, model.predict(rfe.transform(valid_X))))
    f1t.append(f1_score(test_y, model.predict(rfe.transform(test_X))))

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mod

When then inspect the results of all these models on the validation set. You will see most improvement when moving from the single best feature to about the ten best features. After that it becomes a bit random. In fact, you should be aware that when we pick the point with the highest score on the validation set, we are likely to overestimate slightly. We illustrate that in a second plot, the uses a secondary held-out test set. Depending on how the data was randomly sampled, you will see that there is no consistency in the improvement beyond let's say the first 10-20 features. Perhaps this is also a demonstration of Occam's razor, simple models generalize better.

In [ ]:
plt.plot(range(1, valid_X.shape[1]), f1v)
plt.xlabel('#features')
plt.ylabel('f1');

In [ ]:
plt.plot(range(1, valid_X.shape[1]), f1v, label='valid')
plt.plot(range(1, test_X.shape[1]), f1t, label='test')
plt.xlabel('#features')
plt.ylabel('f1')
plt.legend();